# Cognitive Services API

Для многих задач мы с вами можем написать код, который их решает с начала и до конца. Но вместо этого мы иногда можем воспользоваться уже готовыми решениями, и лишь написать код, который задает вопрос и получает ответ. 

Нас с вами как лингвистов интересуют языковые технологии: извлечение ключевых слов, перевод с языка на язык, реферирование, определение эмоциональной нагрузки текста и другие подобные задачи. Многие компании (в их числе, Google, Microsoft, IBM) предоставляют готовые API для этих задач, и попробовать работу этих API можно бесплатно.

Мы рассмотрим одно из таких решений - когнитивные сервисы от Microsoft (Microsoft Cognitive Services), набор API для работы с готовыми машиннообученными моделями от Microsoft. Например, там есть API для работы с изображениями - посылаешь картинку или видео, а в ответ приходит информация о том, что там изображено, а если там изображены люди, то сколько им лет и какие у них эмоции и т.д. И, конечно, есть лингвистические API: обработка речи, извлечение сущностей, спеллчекинг, сентимент-анализ, перевод, извлечение ключевых слов.

Как работают API мы уже с вами поняли на примере ВК: регистрируешься и получаешь токен доступа, посылаешь запрос по специальной ссылке и получаешь ответ в каком-то заданном виде, например, json. Когнитивные сервисы работают точно так же.

### Регистрация
Чтобы получить токен, нужно зарегистрироваться: [например, тут](https://azure.microsoft.com/en-us/try/cognitive-services/) найти Text Analytics API, нажать Create и зарегистрироваться (можно использовать свой аккаунт на GitHub, это занимает 1 минуту). После этого вы получаете ключ для бесплатной работы с Text Analytics API сроком на 90 дней.
Когда у нас есть ключ, можно работать с API. 

Ключ лучше всего положить в отдельный файл, назвать его, например, `conf.py`, и импортировать в основной код:

In [ ]:
KEY = "здесь ваш ключ"

Для работы с API мы будем использовать функцию process_request - она отправляет запрос и получает ответ: если код 200 - возвращает данные, если нет - печатает код ошибки и возвращает None.

In [5]:
import requests 

def process_request(method, url, json, data, headers, params):
    """
    Helper function to process the request
    """
    result = None
    response = requests.request(method, url, json=json, data=data, headers=headers, params=params)

    if response.status_code == 200:

        if 'content-length' in response.headers and int(response.headers['content-length']) == 0:
            result = None
        elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str):
            if 'application/json' in response.headers['content-type'].lower():
                result = response.json() if response.content else None

    else:
        print("Error code: %d" % (response.status_code))
        print(response.json())

    return result

### Text Analytics

Рассмотрим извлечение ключевых слов из текста с помощью API. 

* Демонстрация извлечения ключевых слов с API - [здесь](https://azure.microsoft.com/ru-ru/services/cognitive-services/text-analytics/)

* Описание запросов к API - [здесь](https://westus.dev.cognitive.microsoft.com/docs/services/TextAnalytics.V2.0/operations/56f30ceeeda5650db055a3c6)

* Веб-консоль, для тестрирования запросов - [здесь](https://westus.dev.cognitive.microsoft.com/docs/services/TextAnalytics.V2.0/operations/56f30ceeeda5650db055a3c7/console)

* Quick Start с подробным описанием Text Analytics.V2.0 (sentiment analysis, keywords extraction, language detection) - [здесь](https://docs.microsoft.com/en-us/azure/cognitive-services/text-analytics/quick-start) 

В целом, для работы с апи нам нужно:
* собрать заголовок запроса (headers), в котором мы указываем ключ и описание содержимого (json),
* собрать параметры запроса (params), которые пишутся в URL после знака ? (для Text Analytics это не нужно, но может понадобиться для других когнитивных сервисов),
* собрать json с данными (json), которые будет анализировать сервис,
* собрать нужные файлы (data) (для Text Analytics это не нужно, но может понадобиться для других когнитивных сервисов, в которых мы отправляем файлы, например, при обработке изображений),
* отправить запрос и получить ответ.

Напишем функцию get_key_words, которая получает на вход текст и язык текста, а возвращает массив ключевых слов или None. По умолчанию, функция будет считать, что текст написан на английском языке. Ключевые слова (пока что) доступны для английского, японского, немецкого и испанского языков.

In [1]:
def get_key_words(text, lang="en"):
    """
    Get key phrases from text using Cognitive Services API.
    Args:
        text: string with a text in English
    Returns:
        a list of key phrases or None
    """
    headers = dict()
    headers['Ocp-Apim-Subscription-Key'] = conf.KEY   # conf.KEY stands for YOUR API KEY
    headers['Content-Type'] = 'application/json'
    params = None
    json_data = {
        "documents": [
            {
                "language": lang,
                "id": 'string',
                "text": text
            }
        ]
    }
    data = None
    result = process_request('post', "https://westus.api.cognitive.microsoft.com/text/analytics/v2.0/keyPhrases", 
                             json_data, data, headers, params)
    if result and 'documents' in result:
        return result['documents'][0]["keyPhrases"]
    return None


Теперь попробуем воспользоваться нашей функцией:

In [6]:
text = "The Text Analytics API is a suite of text analytics web services built with Azure Machine Learning. The API can be used to analyze unstructured text for tasks such as sentiment analysis, key phrase extraction and language detection. No training data is needed to use this API; just bring your text data. This API uses advanced natural language processing techniques to deliver best in class predictions."
keywords = get_key_words(text)
print(keywords)

['Text Analytics API', 'text data', 'unstructured text', 'suite of text analytics web services', 'language detection', 'advanced natural language processing techniques', 'sentiment analysis', 'key phrase extraction', 'tasks', 'Azure Machine Learning', 'class predictions', 'training data']


In [7]:
text = "The amygdala is kind of an emotional wreck of a brain structure. It deals with anxiety, sadness, and our responses to fear. There are two amygdalae, and oddly, the left one has been shown to be more balanced, sometimes producing happy feelings in addition to the usual angsty ones, while the right one is always in a bad mood."
keywords = get_key_words(text)
print(keywords)

['emotional wreck', 'usual angsty', 'happy feelings', 'addition', 'responses', 'anxiety', 'sadness', 'amygdalae', 'kind', 'brain structure', 'bad mood']


Sentiment analysis поддерживает большее количество языков, вот их коды: en, es, pt, fr, de, it, nl, no, sv, pl, da, fi, ru, el, tr.

### Другие API

Полный список API - https://westus.dev.cognitive.microsoft.com/docs/services?pageSize=0

Вам может быть интересно:
* Web Language Model API - для работы с языком в интернете (делить текст на слова, посчитать вероятности встречаемости слов, сгенерировать следующие слова)
* Linguistics API - разбор текста (морфология и синтаксис)
* Computer Vision API - обработка изображений (сгенерировать описание изображения, получить теги соответсвующие изображению, найти слова на картинке)

### Задания 
1) Напишите программу, которая получает на вход небольшой текст (например, краткий отзыв), и возвращает sentiment score - насколько этот текст позитивный\негативный. [Sentiment Docs](https://westus.dev.cognitive.microsoft.com/docs/services/TextAnalytics.V2.0/operations/56f30ceeeda5650db055a3c9)

2) Попробуйте написать программу, которая открывает картинку, отправляет ее в [Computer Vision API](https://westus.dev.cognitive.microsoft.com/docs/services/56f91f2d778daf23d8ec6739/operations/56f91f2e778daf14a499e1fa) и возвращает текстовое описание картинки (Describe Image). 
Примеры можно посмотреть в [этом IPython Notebook](https://github.com/Microsoft/Cognitive-Vision-Python/blob/master/Jupyter%20Notebook/Computer%20Vision%20API%20Example.ipynb) (только cv2 и numpy нам не понадобятся, потому что мы ничего не рисуем).